<a href="https://colab.research.google.com/github/Ashubapun/RAG-Implementation/blob/main/RAG_implementation_using_Llama_Index_and_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pypdf

In [3]:
## Embedding
!pip install install sentence_transformers

In [4]:
!pip install llama_index

In [5]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [6]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='861f0a39-4fb5-4633-994b-2be6a7cb3abf', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention is all you need.pdf', 'file_path': '/content/data/Attention is all you need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-01-31', 'last_modified_date': '2024-01-31', 'last_accessed_date': '2024-01-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Us

In [7]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [9]:
!pip install accelerate

In [10]:
!pip install bitsandbytes

In [11]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
!pip install langchain

In [13]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [14]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f510cc0bac0>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f510cc0bac0>, id_func=<function default_id_func at 0x7f5141f812d0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7f510e44b0a0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f510cc0bac0>)

In [15]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)
index

In [16]:
query_engine=index.as_query_engine()

In [17]:
response1=query_engine.query("what is attention is all you need?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [18]:
print(response1)

Attention is All You Need is a phrase used in the context of natural language processing and deep learning, particularly in the field of attention mechanisms. In this context, attention refers to the ability of a model to focus on specific parts of the input when processing it. The phrase "attention is all you need" suggests that attention is the most important aspect of a model's ability to understand and process language.

In the context of the passage, the phrase is used to highlight the importance of attention in the application of the law. The author suggests that the law will never be perfect, but its application should be just, and attention is necessary to achieve this goal.

In summary, "attention is all you need" in the context of the passage refers to the idea that attention is the key to applying the law in a just and fair manner.


In [19]:
response2=query_engine.query("what is bert?")

In [20]:
print(response2)

BERT (Bidirectional Encoder Representations from Transformers) is a pre-training language model developed by Google in 2018. It is trained on a large corpus of text and can be fine-tuned for a wide range of natural language processing tasks, such as question answering, sentiment analysis, and text classification. BERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence, which can be used as input to a downstream task-specific model. BERT has achieved state-of-the-art results on a wide range of natural language processing tasks and has been widely adopted in the NLP community.
